In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


In [2]:
# f = open('/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt','r')
# print(f.read())

In [3]:
train = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
test = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')

In [4]:
train.head(5)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [5]:
### CHoosing the Top 10 Features based on the co-relation with the depende
from statsmodels.stats.outliers_influence import variance_inflation_factor
corr_value = train.corr()
sale_price_corr_val = corr_value.iloc[37:38]
sale_price_corr_val = sale_price_corr_val.loc[:, sale_price_corr_val.ge(0).all()]
sale_price_corr_val = sale_price_corr_val.T
sale_price_corr_val  = sale_price_corr_val.sort_values(by=['SalePrice'],ascending=False)
print(sale_price_corr_val.head(10))

## check any nan values present for those features in train and test
for i in range(1,11):
    #print(train[sale_price_corr_val.index[i]].isnull().values.any())
    print(test[sale_price_corr_val.index[i]].isnull().values.any())
    
## find out those values in test data as train data is perfectly fine

print(test['GrLivArea'].mean())
print(test['GarageCars'].mean())
print(test['GarageArea'].mean())

test['GrLivArea'].fillna(test['GrLivArea'].mean(), inplace=True)
test['GarageCars'].fillna(test['GarageCars'].mean(), inplace=True)
test['GarageArea'].fillna(test['GarageArea'].mean(), inplace=True)

### Find out the most important features in order according to the variance inflation factor 
X = train[['OverallQual', 'GrLivArea', 'GarageCars','GarageArea',
           'TotalBsmtSF','1stFlrSF','FullBath','TotRmsAbvGrd','YearBuilt']]

# VIF dataframe
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns
  
# calculating VIF for each feature
vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                          for i in range(len(X.columns))]
print(vif_data)




#### prediction model ###


x1_train = train[['TotRmsAbvGrd','OverallQual','GrLivArea','GarageCars']]
x1_train = x1_train.values

x1_test = test[['TotRmsAbvGrd','OverallQual','GrLivArea','GarageCars']]
x1_test = x1_test.values

y1_train = train[['SalePrice']]
y1_train = y1_train.values

from sklearn import linear_model
regr = linear_model.LinearRegression()
regr.fit(x1_train, y1_train)
print("Intercept: ", regr.intercept_)
print("Coefficients:")
list(zip(x1_train, regr.coef_))
y_pred= regr.predict(x1_test)
print("Prediction for test set: {}".format(y_pred))

from sklearn import metrics
print('R squared: {:.2f}'.format(regr.score(x1_train,y1_train)*100))


### Scores Can be improved with Cross Val

### Improving score with k fold corss validation
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
lm = linear_model.LinearRegression()
scores = cross_val_score(lm, x1_train, y1_train, scoring='r2', cv=5)
print(scores)

folds = KFold(n_splits = 5, shuffle = True, random_state = 100)
scores = cross_val_score(lm, x1_train, y1_train, scoring='r2', cv=folds)
print(scores)

/tmp/ipykernel_20/3733321342.py:3: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr_value = train.corr()


              SalePrice
SalePrice      1.000000
OverallQual    0.790982
GrLivArea      0.708624
GarageCars     0.640409
GarageArea     0.623431
TotalBsmtSF    0.613581
1stFlrSF       0.605852
FullBath       0.560664
TotRmsAbvGrd   0.533723
YearBuilt      0.522897
False
False
True
True
True
False
False
False
False
False
1486.0459218642907
1.7661179698216736
472.76886145404666
        feature        VIF
0   OverallQual  48.155854
1     GrLivArea  45.131462
2    GarageCars  33.584270
3    GarageArea  29.069732
4   TotalBsmtSF  23.825037
5      1stFlrSF  36.699405
6      FullBath  17.236831
7  TotRmsAbvGrd  56.365797
8     YearBuilt  37.183284
Intercept:  [-90533.16181431]
Coefficients:
Prediction for test set: [[103239.55340917]
 [152446.05084611]
 [164441.4024347 ]
 ...
 [138243.36895748]
 [ 83463.68199429]
 [252804.87297236]]
R squared: 74.00
[0.77921066 0.7425363  0.73666815 0.72541426 0.69259819]
[0.79395467 0.71411484 0.70764402 0.71391173 0.74296739]


In [6]:
x1_train = train[['TotRmsAbvGrd','OverallQual','GrLivArea','GarageCars']]
x1_train = x1_train.values

x1_test = test[['TotRmsAbvGrd','OverallQual','GrLivArea','GarageCars']]
x1_test = x1_test.values

y1_train = train[['SalePrice']]
y1_train = y1_train.values

from sklearn import linear_model
regr = linear_model.LinearRegression()
regr.fit(x1_train, y1_train)
print("Intercept: ", regr.intercept_)
print("Coefficients:")
list(zip(x1_train, regr.coef_))
y_pred= regr.predict(x1_test)
print("Prediction for test set: {}".format(y_pred))

from sklearn import metrics
print('R squared: {:.2f}'.format(regr.score(x1_train,y1_train)*100))


Intercept:  [-90533.16181431]
Coefficients:
Prediction for test set: [[103239.55340917]
 [152446.05084611]
 [164441.4024347 ]
 ...
 [138243.36895748]
 [ 83463.68199429]
 [252804.87297236]]
R squared: 74.00


In [7]:
### Scores Can be improved with Cross Val

### Improving score with k fold corss validation
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
lm = linear_model.LinearRegression()
scores = cross_val_score(lm, x1_train, y1_train, scoring='r2', cv=5)
print(scores)

folds = KFold(n_splits = 5, shuffle = True, random_state = 100)
scores = cross_val_score(lm, x1_train, y1_train, scoring='r2', cv=folds)
print(scores)

[0.77921066 0.7425363  0.73666815 0.72541426 0.69259819]
[0.79395467 0.71411484 0.70764402 0.71391173 0.74296739]
